In [1]:
!pip install tmdbsimple

In [2]:
import json
with open('/Users/marya/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [3]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['api-key']

In [4]:
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] =='US':
            movie_info['certification'] = c['certification']
    return movie_info
            

In [5]:
trial = get_movie_with_rating("tt0848228")
trial

{'adult': False,
 'backdrop_path': '/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 243.768,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [6]:
import os
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data2_2001.csv.gz',
 'final_tmdb_data3_2001.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'Project 3-EDA-TMDB.ipynb',
 'title_akas.csv.gz',
 'title_basics.csv.gz',
 'title_ratings.csv.gz',
 'tmdb_api_results_2001.json']

In [1]:
YEARS_TO_GET = [2000,2001]

In [2]:
from tqdm.notebook import tqdm_notebook
import pandas as pd
for YEAR in tqdm_notebook(YEARS_TO_GET,desc='YEARS',position=0):
        JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
        file_exists = os.path.isfile(JSON_FILE)
        if file_exists == False:
            with open(JSON_FILE,'w') as f:
                json.dump([{'imdb_id':0}],f)

        basics = pd.read_csv("Data/title_basics.csv.gz")
        df = basics.loc[ basics['startYear']==YEAR].copy()
        movie_ids = df['tconst'].copy()
        previous_df = pd.read_json(JSON_FILE)
        movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

        def write_json(new_data, filename):   
            with open(filename,'r+') as file:
                file_data = json.load(file)
                if (type(new_data) == list) & (type(file_data) == list):
                    file_data.extend(new_data)
                else:
                     file_data.append(new_data)
                file.seek(0)
                json.dump(file_data, file)

        for movie_id in tqdm_notebook(movie_ids_to_get,
                                              desc=f'Movies from {YEAR}',
                                              position=1,
                                              leave=True):
                try:
                        temp = get_movie_with_rating(movie_id)
                        write_json(temp,JSON_FILE)
                        time.sleep(0.02)

                except Exception as e:
                        continue

        final_year_df = pd.read_json(JSON_FILE)
        final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/10 [00:00<?, ?it/s]

NameError: name 'FOLDER' is not defined